# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [38]:
# Import Clean City Data
weather = os.path.join('..', 'WheaterPy','Output_Data','Clean_City_Data.csv')
weather_df = pd.read_csv(weather)
weather_df

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (kph),Country,Date
0,Cape Town,-33.9258,18.4232,15.27,91,98,2.68,ZA,1622178009
1,Upernavik,72.7868,-56.1549,3.25,71,100,3.05,GL,1622178518
2,Vernon,50.2581,-119.2691,11.90,52,81,0.89,CA,1622178519
3,Lucea,18.4510,-78.1736,25.48,83,3,2.90,JM,1622178519
4,Cap Malheureux,-19.9842,57.6142,26.37,70,9,3.13,MU,1622178520
...,...,...,...,...,...,...,...,...,...
571,Aloleng,16.1307,119.7824,31.06,68,45,4.17,PH,1622178877
572,Atambua,-9.1061,124.8925,27.51,58,53,1.38,ID,1622178877
573,Masallātah,32.6167,14.0000,25.02,33,68,5.84,LY,1622178877
574,Tangwani,28.2787,68.9972,34.96,25,2,3.78,PK,1622178878


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
# Store latitude and longitude in locations
world_location = weather_df[['Lat', 'Lng']]

# Store humidity
humidity = weather_df['Humidity (%)']

In [40]:
# Plot Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '0px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout, center=(0,0), zoom_level=1.5 )

# Create heat layer
heat_layer = gmaps.heatmap_layer(world_location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='0px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [41]:
fitting_weather = weather_df.loc[(weather_df['Max Temp (C)']>=21) & (weather_df['Max Temp (C)']<=26) & (weather_df['Wind Speed (kph)']<=18) & (weather_df['Cloudiness (%)'] == 0),:]
fitting_weather = fitting_weather.reset_index(drop=True)

In [42]:
fitting_weather

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (kph),Country,Date
0,Bahār,34.9072,48.4414,22.09,43,0,0.95,IR,1622178428
1,Sarāhan,31.5167,77.8000,23.76,25,0,3.02,IN,1622178659
2,Bundaberg,-24.8500,152.3500,25.68,18,0,2.24,AU,1622178672
3,Tsaratanana,-16.7833,47.6500,22.22,63,0,2.67,MG,1622178683
4,Awjilah,29.1081,21.2869,25.57,27,0,3.01,LY,1622178364
5,Santa Cruz Cabrália,-16.2781,-39.0247,21.13,94,0,1.93,BR,1622178691
6,Mackay,-21.1500,149.2000,24.27,40,0,5.45,AU,1622178727
7,Perth,-31.9333,115.8333,22.46,59,0,6.76,AU,1622178503
8,Jalu,29.0331,21.5482,25.52,27,0,3.07,LY,1622178782
9,Behshahr,36.6923,53.5526,23.97,45,0,2.96,IR,1622178804


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
# Store into variable named hotel_df
hotel_df = []

# Params
target_search = 'hotel'
target_radius = 5000
target_type = "lodging"

for city in range(len(fitting_weather['City'])):
    lat = fitting_weather.iloc[city]['Lat']
    lng = fitting_weather.iloc[city]['Lng']
    coords = f'{lat},{lng}'
    params = {
        "location": coords,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.append(response["results"][0]["name"])    
    except:
        hotel_df.append('No lodging nearby')

In [57]:
fitting_weather['Hotel Name'] = hotel_df
fitting_weather.reset_index(drop=True, inplace=True)

In [58]:
fitting_weather

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (kph),Country,Date,Hotel Name,Name
0,Bahār,34.9072,48.4414,22.09,43,0,0.95,IR,1622178428,No lodging nearby,No lodging nearby
1,Sarāhan,31.5167,77.8000,23.76,25,0,3.02,IN,1622178659,Snowflake Sarahan,Snowflake Sarahan
2,Bundaberg,-24.8500,152.3500,25.68,18,0,2.24,AU,1622178672,Burnett Riverside Hotel,Burnett Riverside Hotel
3,Tsaratanana,-16.7833,47.6500,22.22,63,0,2.67,MG,1622178683,No lodging nearby,No lodging nearby
4,Awjilah,29.1081,21.2869,25.57,27,0,3.01,LY,1622178364,No lodging nearby,No lodging nearby
5,Santa Cruz Cabrália,-16.2781,-39.0247,21.13,94,0,1.93,BR,1622178691,Campo Bahia,Campo Bahia
6,Mackay,-21.1500,149.2000,24.27,40,0,5.45,AU,1622178727,Ocean International Hotel,Ocean International Hotel
7,Perth,-31.9333,115.8333,22.46,59,0,6.76,AU,1622178503,"The Ritz-Carlton, Perth","The Ritz-Carlton, Perth"
8,Jalu,29.0331,21.5482,25.52,27,0,3.07,LY,1622178782,Jalu Hotel,Jalu Hotel
9,Behshahr,36.6923,53.5526,23.97,45,0,2.96,IR,1622178804,Ashraf Hotel,Ashraf Hotel


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in fitting_weather.iterrows()]
locations = fitting_weather[["Lat", "Lng"]]

In [68]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map

fig = gmaps.figure(layout=figure_layout, center=(0,0), zoom_level=1 )

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='0px', wi…